In [144]:
import pandas as pd
from datetime import datetime
import pytz

time_zone = pytz.timezone('Asia/Kolkata')
current_time= datetime.now(time_zone)

In [145]:
df_secondary = pd.read_csv('ykb_secondary_pending_abhi.csv')
df_secondary

,shipment_facility_current_name,tracking_id_ekart,shipment_status,bag_type_ph,bag_id,shipment_facility_source_name,bag_facility_source_name,fact_updated_at,bag_facility_source_name_1
0,MotherHub_YKB,FMPP2017961169,sorted,B5,PH-B5-DELYKB-30593238,MotherHub_YKB,BijwasanHub_DEL_PL,2023-11-19T23:50:38.000+05:30,BijwasanHub_DEL_PL
1,MotherHub_YKB,SDLC1026070768,sorted,B5,PH-B5-DELYKB-29433503,MotherHub_YKB,BareillyHub_BRY_PL,2023-11-20T03:49:39.000+05:30,BareillyHub_BRY_PL
2,MotherHub_YKB,FMPP2015747421,sorted,B5,PH-B5-DELYKB-30533329,MotherHub_YKB,MathuraHub_MTH_PL,2023-11-20T03:47:21.000+05:30,MathuraHub_MTH_PL
3,MotherHub_YKB,FMPP2017291645,sorted,ZO,PH-ZO-DELYKB-30533337,MotherHub_YKB,BallabhgarhHub_Del_PL,2023-11-20T02:03:44.000+05:30,BallabhgarhHub_Del_PL
4,MotherHub_YKB,FMPP2010475748,sorted,B5,PH-B5-DELYKB-30419456,MotherHub_YKB,Phase3NoidaHub_DEL_PL,2023-11-20T03:38:32.000+05:30,Phase3NoidaHub_DEL_PL
...,...,...,...,...,...,...,...,...,...
21941,MotherHub_YKB,FMPC3249687132,sorted,B5,PH-B5-DELYKB-30502640,MotherHub_YKB,GurgaonMilestoneHub_DEL_PL,2023-11-20T03:45:07.000+05:30,GurgaonMilestoneHub_DEL_PL
21942,MotherHub_YKB,FMPC3251792220,sorted,B5,PH-B5-DELYKB-30473393,MotherHub_YKB,GurgaonMilestoneHub_DEL_PL,2023-11-20T03:32:28.000+05:30,GurgaonMilestoneHub_DEL_PL
21943,MotherHub_YKB,FMPC3251260872,sorted,B5,PH-B5-DELYKB-30572436,MotherHub_YKB,GurgaonMilestoneHub_DEL_PL,2023-11-20T03:37:09.000+05:30,GurgaonMilestoneHub_DEL_PL
21944,MotherHub_YKB,FMPP2018059189,sorted,B5,PH-B5-DELYKB-30572436,MotherHub_YKB,GurgaonMilestoneHub_DEL_PL,2023-11-20T03:37:09.000+05:30,GurgaonMilestoneHub_DEL_PL


In [146]:
df_secondary_zone_wise  = df_secondary.groupby('bag_type_ph').size()
print(df_secondary_zone_wise)

bag_type_ph
B1        3
B3      141
B4        9
B5    19330
B6      127
B8        9
MR        1
NE        2
ZO     2231
dtype: int64


In [147]:
df_secondary_zo = df_secondary[df_secondary['bag_type_ph'] == "ZO"]
df_secondary_zo['bag_facility_source_name'].fillna("Not Found" , inplace=True)
df_secondary_zo_ph  = df_secondary_zo[df_secondary_zo['bag_facility_source_name'].str.contains("DEL_PL|Bhiwadi|Bhiwani|Bilaspur")]
df_secondary_zo_sph = df_secondary_zo[~df_secondary_zo.index.isin(df_secondary_zo_ph.index)]

df_secondary_zo_ph1 = df_secondary_zo_ph.groupby('bag_facility_source_name').size()
df_secondary_zo_sph1 = df_secondary_zo_sph.groupby('bag_facility_source_name').size()

print(f"Secondary Pending ZO PH: {sum(df_secondary_zo_ph1)}")
print(f"Secondary Pending ZO SPH: {sum(df_secondary_zo_sph1)}")
print(f"Secondary Total Pending ZO: {sum(df_secondary_zo_ph1) + sum(df_secondary_zo_sph1)} ")


Secondary Pending ZO PH: 1848
Secondary Pending ZO SPH: 383
Secondary Total Pending ZO: 2231 


C:\Users\abhishek.h1\AppData\Local\Temp\ipykernel_20364\200542944.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_secondary_zo['bag_facility_source_name'].fillna("Not Found" , inplace=True)


In [148]:
df_secondary_b5 = df_secondary[df_secondary['bag_type_ph'] == "B5"]
df_secondary_b5['bag_facility_source_name'].fillna("Not Found" , inplace=True)

df_secondary_b5_ph = df_secondary_b5[df_secondary_b5['bag_facility_source_name'].str.contains("DEL_PL|BHiwadi|Bhiwani|Bilaspur")]
df_secondary_b5_sph = df_secondary_b5[~df_secondary_b5.index.isin(df_secondary_b5_ph.index)]

df_secondary_b5_ph1 = df_secondary_b5_ph.groupby('bag_facility_source_name').size()
df_secondary_b5_sph1 = df_secondary_b5_sph.groupby('bag_facility_source_name').size()

print(f"Secondary Pending B5 PH: {sum(df_secondary_b5_ph1)}")
print(f"Secondary Pending B5 SPH: {sum(df_secondary_b5_sph1)}")
print(f"Secondary Total Pending B5: {sum(df_secondary_b5_ph1) + sum(df_secondary_b5_sph1)} ")

Secondary Pending B5 PH: 16671
Secondary Pending B5 SPH: 2659
Secondary Total Pending B5: 19330 


C:\Users\abhishek.h1\AppData\Local\Temp\ipykernel_20364\180343183.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_secondary_b5['bag_facility_source_name'].fillna("Not Found" , inplace=True)


In [152]:
def calculate_and_categorize_time(dataframe, column_name, current_time  , bucket_name):
    df_time_wise = pd.to_datetime(dataframe[column_name])
    df_hour_wise = (current_time - df_time_wise) / pd.Timedelta(hours=1)
    df_hour_wise = df_hour_wise.round().astype(int)
    
    def categorize_time(hour_diff):
        if hour_diff > 48:
            return '> 48 hours'
        elif hour_diff > 24 and hour_diff < 48:
            return '24-48 hours'
        elif hour_diff > 11:
            return '12-24 hours'
        else:
            return '< 12 hours'
    
    dataframe["hour_diff"] = df_hour_wise
    dataframe[bucket_name] = df_hour_wise.apply(categorize_time)
    data = dataframe[bucket_name].value_counts()
    return data


zo_ph_secondary_total = calculate_and_categorize_time(df_secondary_zo_ph , 'fact_updated_at' , current_time , "ZO Secondary Pending PH: ")
print(zo_ph_secondary_total)

zo_sph_secondary_total = calculate_and_categorize_time(df_secondary_zo_sph , 'fact_updated_at' , current_time , "ZO Secondary Pending SPH: ")
print(zo_sph_secondary_total)

print(f"Total ZO Secondary Pending:    {sum(zo_ph_secondary_total) + sum(zo_sph_secondary_total)}")


b5_ph_secondary_total = calculate_and_categorize_time(df_secondary_b5_ph , 'fact_updated_at' , current_time , "B5 Secondary Pending PH: ")
print(b5_ph_secondary_total)

b5_sph_secondary_total = calculate_and_categorize_time(df_secondary_b5_sph , 'fact_updated_at' , current_time , "B5 Secondary Pending SPH: ")
print(b5_sph_secondary_total)

print(f"Total B5 Secondary Pending:  {sum(b5_ph_secondary_total) + sum(b5_sph_secondary_total)} ")

ZO Secondary Pending PH: 
< 12 hours     1423
> 48 hours      266
24-48 hours      81
12-24 hours      78
Name: count, dtype: int64
ZO Secondary Pending SPH: 
< 12 hours     295
> 48 hours      51
24-48 hours     21
12-24 hours     16
Name: count, dtype: int64
Total ZO Secondary Pending:    2231
B5 Secondary Pending PH: 
< 12 hours     16391
> 48 hours       135
12-24 hours      119
24-48 hours       26
Name: count, dtype: int64
B5 Secondary Pending SPH: 
< 12 hours     2569
12-24 hours      46
> 48 hours       37
24-48 hours       7
Name: count, dtype: int64
Total B5 Secondary Pending:  19330 


C:\Users\abhishek.h1\AppData\Local\Temp\ipykernel_20364\2868395101.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["hour_diff"] = df_hour_wise
C:\Users\abhishek.h1\AppData\Local\Temp\ipykernel_20364\2868395101.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[bucket_name] = df_hour_wise.apply(categorize_time)
C:\Users\abhishek.h1\AppData\Local\Temp\ipykernel_20364\2868395101.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi